Read the `hdf5` file, reduce the order and train a neural network

In [121]:
import h5py
import numpy as np

def sliceDataAlongAxis(data, fractions, axis):
    data_size = data.shape[axis]
    fractions_ = np.zeros_like(fractions, dtype=int)

    total_size = 0
    for i, fraction in enumerate(fractions):
        total_size += int(data_size*fraction)
    remain = data_size-total_size

    slices = ()
    for i, fraction in enumerate(fractions):
        fractions_[i] = int(data_size*fraction)
        if i > 0:
            fractions_[i] += fractions_[i-1]
            slice = data.take(range(fractions_[i-1], fractions_[i]), axis)

        else:
            slice = data.take(range(0, fractions_[i]+remain), axis)

        slices += (slice,)

    return slices

def get_snapshots(h5_file_path, variables):
    # get snapshots from h5 file
    snapshots = []
    indexes = {}
    with h5py.File(h5_file_path, 'r') as h5_file:
        for var in variables:
            snapshots.append( h5_file[var][()])
            end_idx = start_idx + h5_file[var][()].shape[0]
            indexes[var] = np.arange(start_idx, end_idx)
            start_idx = end_idx
    return np.vstack(snapshots), indexes

In [122]:
class dataHandler:
    def __init__(self, file, variables):
        self.file = file
        self.variables = variables

        self.stack()

    def stack(self):
        with h5py.File(self.file, 'r') as f:
            self.h5 = f
            self.data = np.vstack([f[var][()] for var in self.variables])

            self.indexes = {}
            start_idx = 0
            for var in self.variables:
                try:
                    end_idx = start_idx + f[var][()].shape[0]
                except:
                    # f[var] is a scalar
                    end_idx = start_idx + 1
                self.indexes[var] = np.arange(start_idx, end_idx)
                start_idx = end_idx

            if 'meshfile' in f.keys():
                self.meshfile = f['meshfile'][()]

    def get_variable(self, variable=None, data=None,):
        if data is None:
            data = self.data

        return data[self.indexes[variable], :]
    
    def split_train_validation_test(self, fractions):
        self.train , self.validation, self.test = sliceDataAlongAxis(
            self.data, fractions, 1)

In [123]:
from svd_dataset import SVD
import tensorflow as tf

class ROM(object):
    def __init__(self, dataset, rank=None):

        self.svd = SVD(dataset)
        
        self.bounds = [0,1]
        
        self.svd.normalize(bounds=self.bounds)
        self.svd.subtractMean()
        self.svd.SVD()
        self.setRank(rank)

    def setRank(self, rank):
        self.rank = rank
        self.svd.setRank(self.rank)

    def setEnergy(self, e):
        self.energyPreserved = e
        self.rank = self.svd.findRank(self.energyPreserved)
        self.svd.setRank(self.rank)
    
    def reduce(self, snapshot):
        snapshot,min,max = self.svd.normalize(snapshot, self.bounds, self.svd.min, self.svd.max)
        snapshot,mean = self.svd.subtractMean(snapshot, self.svd.mean)
        L = (self.svd.u.T @ snapshot).T
        return L
    
    @property
    def data(self):
        return self.svd.L

    @property
    def L(self):
        return self.svd.L
    
    def reconstruct(self, input):
        return self.svd.reconstruct(input)


class NeuralNetwork:
    def __init__(self, 
                 layers, 
                 activation_function = 'tanh', 
                 optimizer = tf.keras.optimizers.Adam()):
        
        self.name = 'neural_network'
        
        self.nn = self.get_model(layers, activation_function, optimizer)    
    
    def get_model ( self, layers, activation_function, optimizer ):
        # Input layer
        ph_input = tf.keras.Input( shape =( layers[0] ,) ,name='input_placeholder')
        # Hidden layers
        hidden_layer = ph_input
        for num_neurons in layers[1:-1]:
            hidden_layer = tf.keras.layers.Dense ( num_neurons , activation = activation_function)( hidden_layer )

        # Output layer
        output = tf.keras.layers.Dense ( layers[-1] , activation ='linear',name='output_value')( hidden_layer)

        model = tf.keras.Model ( inputs =[ ph_input ], outputs =[ output ])
        # Compilation
        model.compile ( optimizer = optimizer , loss ={ 'output_value': 'mean_squared_error'})

        return model
    
    def fit(self, 
        train_input, 
        train_target, 
        validation_input=None, 
        validation_target=None, 
        epochs=1000, 
        batch_size=16  ):
        
        if validation_input is not None and validation_target is not None:       
            validation_tuple = (validation_input, validation_target)

        self.history = self.nn.fit(train_input, train_target, epochs=epochs, batch_size=batch_size, validation_data=validation_tuple)

        return self.nn, self.history
    
    def predict(self, input):
        return self.nn.predict(input)
    
    def save(self, file):
        self.nn.save(file)

    def load(self, file):
        self.nn = tf.keras.models.load_model( file )


In [124]:
lf_data_handler = dataHandler(
    '/home/ppiper/Dropbox/local/ihtc_repository/data/doe_30/q1d.h5', 
    ['Thickness', 'CP3_y', 'Temperature', 'Pressure', 'Mach'])


In [125]:
hf_data_handler = dataHandler(
    '/home/ppiper/Dropbox/local/ihtc_repository/data/doe_30/su2.h5', 
    ['Thickness', 'CP3_y', 'Temperature', 'Pressure', 'Mach', 'Temperature_Solid', 'Temperature_Solid_INNERWALL', 'Heat_Flux_UPPER_WALL'])

In [127]:
reconstructed_mesh =pv.read('/home/ppiper/Dropbox/local/ihtc_repository/data/single_run/SU2/outputs/reconstructed.vtm')

reconstructed_mesh['Zone 0 (Comp. Fluid)']['Internal']['Internal']['Pressure']= hf_data_handler.get_variable('Pressure')[:,0] #fr.reconstruct(fr.lf_solution_handler.data, 'Pressure')

reconstructed_mesh['Zone 0 (Comp. Fluid)']['Internal']['Internal']['Temperature']=hf_data_handler.get_variable('Temperature')[:,0]#fr.reconstruct(fr.lf_solution_handler.data, 'Temperature')

reconstructed_mesh['Zone 0 (Comp. Fluid)']['Internal']['Internal']['Mach']=get_su2_variable('/home/ppiper/Dropbox/local/ihtc_repository/data/doe_30/11/SU2/outputs/cht_setupSU2.vtm','Mach')
#hf_data_handler.get_variable('Mach')[:,0]#fr.reconstruct(fr.lf_solution_handler.data, 'Mach')

reconstructed_mesh['Zone 1 (Solid Heat)']['Internal']['Internal']['Temperature']=hf_data_handler.get_variable('Temperature_Solid')[:,0]#fr.reconstruct(fr.lf_solution_handler.data, 'Temperature_Solid')

reconstructed_mesh['Zone 1 (Solid Heat)']['Boundary']['INNERWALL']['Temperature']=hf_data_handler.get_variable('Temperature_Solid_INNERWALL')[:,0] #fr.reconstruct(fr.lf_solution_handler.data, 'Temperature_Solid_INNERWALL')

#reconstructed_mesh['Zone 0 (Comp. Fluid)']['Boundary']['UPPER_WALL']['Heat_Flux']=hf_data_handler.get_variable('Heat_Flux_UPPER_WALL')[:,0]#fr.reconstruct(fr.lf_solution_handler.data, 'Heat_Flux_UPPER_WALL')


reconstructed_mesh.save('/home/ppiper/Dropbox/local/ihtc_repository/data/single_run/SU2/outputs/reconstructed.vtm')


TypeError: unsupported operand type(s) for /: 'str' and 'str'

In [79]:
lf_data_handler.split_train_validation_test([0.8, 0.1, 0.1])
hf_data_handler.split_train_validation_test([0.8, 0.1, 0.1])

In [80]:
rank = 10
lf_rom = ROM(lf_data_handler.train, rank)
hf_rom = ROM(hf_data_handler.train, rank)


In [81]:
surrogate = NeuralNetwork(
    layers=[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10], activation_function='tanh',
    optimizer=tf.keras.optimizers.Adam()
    )

In [82]:
surrogate.load('/home/ppiper/Dropbox/local/ihtc_repository/data/doe_30/trained_nn')

In [83]:
#surrogate.save('/home/ppiper/Dropbox/local/ihtc_repository/data/doe_30/trained_nn')

In [84]:
#surrogate.fit(train_input= lf_rom.data,
#              train_target= hf_rom.data,
#              validation_input= lf_rom.reduce(lf_data_handler.validation),
#              validation_target= hf_rom.reduce(hf_data_handler.validation),
#              epochs=300)

In [85]:
import configparser
import pathlib
import pandas as pd

import pyvista as pv
def clear_vtm(solution_file, output_file):
    multiblock = pv.read(solution_file)
    for i in range(len(multiblock)):
        for j in range(len(multiblock[i])):
            for k in range(len(multiblock[i][j])):
                multiblock[i][j][k].clear_point_data()
    multiblock.save(output_file)

    return pv.read(output_file)

def read_config_file(config_file):
        config = CaseConfigParser()
        config.read(config_file)

        # convert all path to the absolute
        for each_section in config.sections():
            for (each_key, each_val) in config.items(each_section):
                value = pathlib.Path(config[each_section][each_key])
                if value.is_file() or value.is_dir():
                    config[each_section][each_key] = f"{value.resolve()}"
        return config

def get_q1d_variable(solution_path, variable_name):
    solution_subpath = solution_path / 'Q1D' / 'outputs'
    name_to_variable_conversion = {
        'Pressure': np.loadtxt(solution_subpath / 'p.txt'),
        'Temperature': np.loadtxt(solution_subpath / 'T.txt') ,
        'Mach' : np.loadtxt(solution_subpath / 'M.txt'),
    }
    return name_to_variable_conversion[variable_name]


def get_block_recursive(block, name_index):
    for n in name_index:
        block = block[n]
    return block

def set_block_recursive(block, name_index, value):
    for n in name_index[:-1]:
        block = block[n]
    block[name_index[-1]] = value

hf_name_to_variable_conversion =  {
        'Pressure' : ['Zone 0 (Comp. Fluid)','Internal','Internal','Pressure'] ,
        'Temperature' : ['Zone 0 (Comp. Fluid)','Internal','Internal','Temperature'],
        'Mach' : ['Zone 0 (Comp. Fluid)','Internal','Internal','Mach'],
        'Temperature_Solid' : ['Zone 1 (Solid Heat)', 'Internal', 'Internal', 'Temperature'],
        'Temperature_Solid_INNERWALL' : ['Zone 1 (Solid Heat)', 'Boundary', 'INNERWALL', 'Temperature'],
        'Heat_Flux_UPPER_WALL' : ['Zone 0 (Comp. Fluid)', 'Boundary','UPPER_WALL', 'Heat_Flux'] ,
    }

def set_su2_variable(solution_path, variable_name, variable):
    solution_path = pathlib.Path(solution_path)
    #solution_subpath = solution_path / 'SU2' / 'outputs' / "cht_setupSU2.vtm"
    block = pv.read(solution_path)

    set_block_recursive(block, hf_name_to_variable_conversion[variable_name], variable)

def get_su2_variable(solution_path, variable_name):
    solution_subpath = solution_path / 'SU2' / 'outputs' / "cht_setupSU2.vtm"
    block = pv.read(solution_subpath)
    
    return  get_block_recursive(block, hf_name_to_variable_conversion[variable_name])

class CaseConfigParser(configparser.ConfigParser):
    def optionxform(self, optionstr):
        return optionstr
        
class FlowReconstruction:
    def __init__(self):
        pass
    
    def set_lf_model(self, lf_model):

        self.lf_model = lf_model

    def set_hf_model(self, hf_model):
        # set the high fidelity model function to run the solver
        self.hf_model = hf_model

    def run_lf_model(self, **other_params):

        config = read_config_file(self.config_file)        

        LF_PARAMS = dict(config['LF_PARAMS'])

        self.lf_model(rootfile = self.rootfile, **LF_PARAMS, **other_params)

        q1d_variables = [
            'Pressure',
            'Temperature',
            'Mach' 
        ]

        h5_file = self.rootfile + 'lf.h5'

        self.genHDF5(dataset_path = self.rootfile,
                     variables = q1d_variables,
                     outputfile = h5_file,
                     variable_getter = get_q1d_variable,
                     doe_variables = other_params,
        )

        self.lf_solution_handler = dataHandler(h5_file, list(other_params.keys())+ q1d_variables)

    def gen_hf_mesh(self, **other_params):
        # read configuration file containing HF_PARAMS
        config = read_config_file(self.config_file)
        HF_PARAMS = dict(config['HF_PARAMS'])

        # other_parmas is a dict with design variables
        model = self.hf_model(rootfile = self.rootfile, only_generate_mesh=True, **HF_PARAMS, **other_params)

        su2_variables = [
            'Pressure', 
            'Temperature', 
            'Mach', 
            'Temperature_Solid', 
            'Temperature_Solid_INNERWALL', 
            'Heat_Flux_UPPER_WALL'
        ]

        h5_file = self.rootfile + 'hf.h5'

        self.genHDF5(dataset_path = self.rootfile,
                     variables = su2_variables,
                     outputfile = h5_file,
                     variable_getter = get_su2_variable,
                     doe_variables = other_params,
        )

        self.hf_solution_handler = dataHandler(h5_file, list(other_params.keys())+ su2_variables)

        self.reconstructed_file = pathlib.Path(model.su2outfilepath) / "reconstructed.vtm"

        clear_vtm(model.solution_file, self.reconstructed_file)

        # interv name_to_variable_conversion
        #for variable_name in su2_variables:
        #    variable = self.reconstruct(self.lf_solution_handler.data, variable_name)
        #    set_su2_variable(self.reconstructed_file, variable_name, variable)

    def genHDF5(self, dataset_path, variables, outputfile, variable_getter, doe_variables):
        with h5py.File(outputfile, 'w') as h5file:
            for key, val in doe_variables.items():
                h5file[key] = val

            for variable_name in variables:
                solution_path = pathlib.Path(dataset_path )
                variable_data = variable_getter(solution_path, variable_name)

                h5file.create_dataset(variable_name,shape=(len(variable_data), 1))
                    
                h5file[variable_name][...,0] = variable_data

    def gen_doe(self, doe_config):
        self.DoE = DesignOfExperiment()
        self.DoE.set_config(doe_config)
        self.DoE.gen_doe()

    def set_fr(self, 
                 lf_rom = None, 
                 hf_rom = None, 
                 surrogate = None,
                 lf_data_handler = None,
                 hf_data_handler = None, 
                 rootfile = None, 
                 config_file = None):
        # reciev data handler for low fidelity and high fidelity models
        #self.lf_data = lf_data
        #self.hf_data = hf_data
        self.lf_rom = lf_rom
        self.hf_rom = hf_rom
        self.surrogate = surrogate
        self.lf_data_handler = lf_data_handler
        self.hf_data_handler = hf_data_handler
        self.rootfile = rootfile
        self.config_file = config_file
        pass
    
    def save(self, file):
        self.surrogate_model.save(file)
    
    def load(self, file):
        self.surrogate_model = self.surrogate_model.load(file)

    def reconstruct(self, lf_data, variable=None):
        if len(lf_data.shape) == 1:
            lf_data = lf_data[:,None]
                
        self.lf_data_reduced = self.lf_rom.reduce(lf_data)
        self.hf_data_predicted = self.surrogate.predict(self.lf_data_reduced)
        self.hf_data_reconstructed = self.hf_rom.reconstruct(self.hf_data_predicted)

        if variable is not None:
            self.hf_data_reconstructed = self.hf_data_handler.get_variable(data=self.hf_data_reconstructed, variable=variable)
        
        return self.hf_data_reconstructed
    
    def fit(self, 
            input_data, 
            target_data, 
            input_validation=None, 
            target_validation=None):
        
        self.input_data = input_data
        self.target_data = target_data

        if self.lf_rom is not None:
            self.input_data = self.lf_rom.reduce(input_data)

        if self.hf_rom is not None:
            self.target_data = self.hf_rom.reduce(target_data)

        self.surrogate.fit(
            self.input_data, 
            self.target_data, 
            self.input_validation, 
            self.target_validation)

In [86]:
#from flowrec import FlowReconstruction
from models_parametric_shape import lf_model, get_q1d_variable, hf_model

fr = FlowReconstruction()

fr.set_fr(
    lf_rom    = lf_rom, 
    hf_rom    = hf_rom, 
    surrogate = surrogate, 
    lf_data_handler = lf_data_handler,
    hf_data_handler = hf_data_handler,
    rootfile = '../data/single_run/',
    config_file = '/home/ppiper/Dropbox/local/ihtc_repository/src/doe_30.cfg'
)

fr.set_lf_model( lf_model )
fr.set_hf_model( hf_model )

In [87]:
fr.run_lf_model(Thickness=0.004, CP3_y=-0.0025)

['/home/ppiper/Dropbox/local/ihtc_repository/src/eulerQ1D', '../data/single_run/Q1D/inputs/setupQ1D.txt']
################################################################################
                           -*- Q1D Euler Solver -*-
Eigen 3.3.7
Allan Moreira de Carvalho
################################################################################
                           -*- Setup Information -*-
################################################################################
# Domain x-coordinates at cell faces (no need for ghost cells)
../data/single_run/Q1D/inputs/xn.txt
# Area distributuin at cell faces (no need for ghost cells)
../data/single_run/Q1D/inputs/Sn.txt
# Inlet Total Pressure [Pa]
800000.0
# Inlet Total Temperature [K]
600.0
# Inlet Mach Number
0.01
# Outlet Static Pressure [Pa]
101000.0
# Gas constant [J/kg/K]
287.0491267875601
# Specific heat ratio
1.3781736037783794
# Maximum number of iterations 
50000
# Interval to print iterations 
1000
# CFL number 
0.

In [89]:
fr.reconstruct(fr.lf_solution_handler.data).shape

1/1 [==============================] - 0s 51ms/step


(252842, 1)

In [90]:
hf_data_handler.indexes['Heat_Flux_UPPER_WALL']

array([252690, 252691, 252692, 252693, 252694, 252695, 252696, 252697,
       252698, 252699, 252700, 252701, 252702, 252703, 252704, 252705,
       252706, 252707, 252708, 252709, 252710, 252711, 252712, 252713,
       252714, 252715, 252716, 252717, 252718, 252719, 252720, 252721,
       252722, 252723, 252724, 252725, 252726, 252727, 252728, 252729,
       252730, 252731, 252732, 252733, 252734, 252735, 252736, 252737,
       252738, 252739, 252740, 252741, 252742, 252743, 252744, 252745,
       252746, 252747, 252748, 252749, 252750, 252751, 252752, 252753,
       252754, 252755, 252756, 252757, 252758, 252759, 252760, 252761,
       252762, 252763, 252764, 252765, 252766, 252767, 252768, 252769,
       252770, 252771, 252772, 252773, 252774, 252775, 252776, 252777,
       252778, 252779, 252780, 252781, 252782, 252783, 252784, 252785,
       252786, 252787, 252788, 252789, 252790, 252791, 252792, 252793,
       252794, 252795, 252796, 252797, 252798, 252799, 252800, 252801,
      

In [91]:
original_mesh  = pv.read('/home/ppiper/Dropbox/local/ihtc_repository/data/single_run/SU2/outputs/cht_setupSU2.vtm')
original_mesh['Zone 0 (Comp. Fluid)']['Internal']['Internal']['Pressure'].shape

(69300,)

In [92]:
original_mesh['Zone 0 (Comp. Fluid)']['Internal']['Internal']

UnstructuredGrid (0x7f00018dc400)
  N Cells:    68761
  N Points:   69300
  X Bounds:   1.245e-03, 1.504e-01
  Y Bounds:   -6.282e-02, 6.282e-02
  Z Bounds:   0.000e+00, 0.000e+00
  N Arrays:   14

In [93]:
fr.gen_hf_mesh(Thickness=0.004, CP3_y=-0.0025)


Info    : Running '/home/ppiper/Dropbox/local/ihtc_repository/src/gmsh /home/ppiper/Dropbox/local/ihtc_repository/data/single_run/SU2/inputs/setupSU2.geo -0 -2 -format su2 -o /home/ppiper/Dropbox/local/ihtc_repository/data/single_run/SU2/inputs/setupSU2.su2' [Gmsh 4.9.1, 1 node, max. 1 thread]
Info    : Started on Mon Apr 24 16:33:16 2023
Info    : Reading '/home/ppiper/Dropbox/local/ihtc_repository/data/single_run/SU2/inputs/setupSU2.geo'...
Info    : Done reading '/home/ppiper/Dropbox/local/ihtc_repository/data/single_run/SU2/inputs/setupSU2.geo'
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 50%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.0427018s, CPU 0.08746s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Transfinite)
Info    : Done meshing 2D (Wall 0.0192753s, CPU 0.02267s)
Info    : 71296 nodes 71837 elements
Info    : Writing '/home/ppiper/Dropbox/lo

Info    : Running '/home/ppiper/Dropbox/local/ihtc_repository/src/gmsh -3 -smooth 5 -optimize /home/ppiper/Dropbox/local/ihtc_repository/data/single_run/SU2/inputs/solid_setupSU2.geo -0 -2 -format su2 -o /home/ppiper/Dropbox/local/ihtc_repository/data/single_run/SU2/inputs/solid_setupSU2.su2' [Gmsh 4.9.1, 1 node, max. 1 thread]
Info    : Started on Mon Apr 24 16:33:17 2023
Info    : Reading '/home/ppiper/Dropbox/local/ihtc_repository/data/single_run/SU2/inputs/solid_setupSU2.geo'...
Info    : Done reading '/home/ppiper/Dropbox/local/ihtc_repository/data/single_run/SU2/inputs/solid_setupSU2.geo'
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Nurb)
Info    : [ 20%] Meshing curve 2 (Extruded)
Info    : [ 30%] Meshing curve 3 (Extruded)
Info    : [ 40%] Meshing curve 4 (Extruded)
Info    : [ 50%] Meshing curve 5 (Nurb)
Info    : [ 70%] Meshing curve 6 (Extruded)
Info    : [ 80%] Meshing curve 7 (Extruded)
Info    : [ 90%] Meshing curve 8 (Extruded)
Info    : Done meshing 1D (Wal

Info    : [ 50%] Meshing surface 9 (Extruded)
Info    : Done meshing 2D (Wall 0.337461s, CPU 0.437951s)
Info    : 48512 nodes 49146 elements
Info    : Writing '/home/ppiper/Dropbox/local/ihtc_repository/data/single_run/SU2/inputs/solid_setupSU2.msh'...
Info    : Done writing '/home/ppiper/Dropbox/local/ihtc_repository/data/single_run/SU2/inputs/solid_setupSU2.msh'
Info    : Stopped on Mon Apr 24 16:33:18 2023 (From start: Wall 0.564475s, CPU 0.737985s)
Info    : Running '/home/ppiper/Dropbox/local/ihtc_repository/src/gmsh -3 -smooth 5 -optimize /home/ppiper/Dropbox/local/ihtc_repository/data/single_run/SU2/inputs/solid_setupSU2.geo -0 -2 -format msh -o /home/ppiper/Dropbox/local/ihtc_repository/data/single_run/SU2/inputs/solid_setupSU2.vtk' [Gmsh 4.9.1, 1 node, max. 1 thread]
Info    : Started on Mon Apr 24 16:33:18 2023
Info    : Reading '/home/ppiper/Dropbox/local/ihtc_repository/data/single_run/SU2/inputs/solid_setupSU2.geo'...
Info    : Done reading '/home/ppiper/Dropbox/local/ihtc

Info    : [ 50%] Meshing surface 9 (Extruded)
Info    : Done meshing 2D (Wall 0.416214s, CPU 0.557257s)
Info    : 48512 nodes 49146 elements
Info    : Writing '/home/ppiper/Dropbox/local/ihtc_repository/data/single_run/SU2/inputs/solid_setupSU2.vtk'...
Info    : Done writing '/home/ppiper/Dropbox/local/ihtc_repository/data/single_run/SU2/inputs/solid_setupSU2.vtk'
Info    : Stopped on Mon Apr 24 16:33:19 2023 (From start: Wall 0.66646s, CPU 0.913197s)
Input files:
1. setupSU2
2. solid_setupSU2
Multi-zone mesh written to file: /home/ppiper/Dropbox/local/ihtc_repository/data/single_run/SU2/inputs/multizone.su2

-------------------------------------------------------------------------
|    ___ _   _ ___                                                      |
|   / __| | | |_  )   Release 7.4.0 "Blackbird"                         |
|   \__ \ |_| |/ /                                                      |
|   |___/\___//___|   Suite (Computational Fluid Dynamics Code)         |
|            

In [48]:
fr.lf_solution_handler.get_variable('Pressure')

(401, 1)

In [52]:
fr.reconstruct(fr.lf_solution_handler.data, 'Pressure')

1/1 [==============================] - 0s 43ms/step


(69300, 1)

In [107]:
reconstructed_mesh =pv.read('/home/ppiper/Dropbox/local/ihtc_repository/data/single_run/SU2/outputs/reconstructed.vtm')

reconstructed_mesh['Zone 0 (Comp. Fluid)']['Internal']['Internal']['Pressure']=fr.reconstruct(fr.lf_solution_handler.data, 'Pressure')

reconstructed_mesh['Zone 0 (Comp. Fluid)']['Internal']['Internal']['Temperature']=fr.reconstruct(fr.lf_solution_handler.data, 'Temperature')

reconstructed_mesh['Zone 0 (Comp. Fluid)']['Internal']['Internal']['Mach']=fr.reconstruct(fr.lf_solution_handler.data, 'Mach')

reconstructed_mesh['Zone 1 (Solid Heat)']['Internal']['Internal']['Temperature']=fr.reconstruct(fr.lf_solution_handler.data, 'Temperature_Solid')

reconstructed_mesh['Zone 1 (Solid Heat)']['Boundary']['INNERWALL']['Temperature']=fr.reconstruct(fr.lf_solution_handler.data, 'Temperature_Solid_INNERWALL')

#reconstructed_mesh['Zone 0 (Comp. Fluid)']['Boundary']['UPPER_WALL']['Heat_Flux']=fr.reconstruct(fr.lf_solution_handler.data, 'Heat_Flux_UPPER_WALL')


reconstructed_mesh.save('/home/ppiper/Dropbox/local/ihtc_repository/data/single_run/SU2/outputs/reconstructed.vtm')


1/1 [==============================] - 0s 60ms/step


In [35]:
import pathlib

class FOM:
    def __init__(self):
        pass

    def set_run_model_function(self, run_model_function):
        self.run_model_function = run_model_function

    def set_params(self, params):
        self.params = params

    def run(self):
        self.instance = self.run_model_function(**self.params)

    def get_variable(self, variable):
        solution_path = pathlib.Path(self.params['rootfile']).absolute()
        return self.variable_getter(solution_path, variable)

    def set_variable_getter(self, variable_getter):
        self.variable_getter = variable_getter

    def get_snapshot(self, variables, design_variables=None):
        solution_path = pathlib.Path(self.params['rootfile']).absolute()
        variable_getter = self.variable_getter
        data = {}
        for dvariable in design_variables:
            data[dvariable] = self.params[dvariable]
        for variable_name in variables:
            variable_data = variable_getter(solution_path, variable_name)    
            data[variable_name] = variable_data

        return np.hstack([data[var] for var in data.keys()])
        

In [36]:
from models_parametric_shape import lf_model

lf_params = dict(
    Nx = 400,
    EULER_Q1D_SOLVER = './eulerQ1D',
    baselineCP = './baselineCP.txt',
    T0in = 600.0,
    p0in = 8.0e5,
    CP3_y =  -0.0024504689702769863,
    Thickness = 0.004209215007771409,
    rootfile = '../data/single_run/'
)

def get_q1d_variable(solution_path, variable_name):
    solution_subpath = solution_path / 'Q1D' / 'outputs'
    input_subpath = solution_path / 'Q1D' / 'inputs'
    name_to_variable_conversion = {
        'Pressure': np.loadtxt(solution_subpath / 'p.txt'),
        'Temperature': np.loadtxt(solution_subpath / 'T.txt') ,
        'Mach' : np.loadtxt(solution_subpath / 'M.txt'),
        'x'    : np.loadtxt(input_subpath / 'xn.txt'),
        'Area'    : np.loadtxt(input_subpath / 'Sn.txt'),
    }
    return name_to_variable_conversion[variable_name]

lf_fom = FOM()
lf_fom.set_run_model_function(lf_model)
lf_fom.set_params(lf_params)
lf_fom.set_variable_getter( get_q1d_variable )
lf_fom.run()

['./eulerQ1D', '../data/single_run/Q1D/inputs/setupQ1D.txt']
################################################################################
                           -*- Q1D Euler Solver -*-
Eigen 3.3.7
Allan Moreira de Carvalho
################################################################################
                           -*- Setup Information -*-
################################################################################
# Domain x-coordinates at cell faces (no need for ghost cells)
../data/single_run/Q1D/inputs/xn.txt
# Area distributuin at cell faces (no need for ghost cells)
../data/single_run/Q1D/inputs/Sn.txt
# Inlet Total Pressure [Pa]
800000.0
# Inlet Total Temperature [K]
600.0
# Inlet Mach Number
0.01
# Outlet Static Pressure [Pa]
101000.0
# Gas constant [J/kg/K]
287.0491267875601
# Specific heat ratio
1.3781736037783794
# Maximum number of iterations 
50000
# Interval to print iterations 
1000
# CFL number 
0.15
# Convergence criteria 
1e-08
# Time integ

In [37]:
hf_fom.params

NameError: name 'hf_fom' is not defined

In [ ]:
from models_parametric_shape import hf_model
import pyvista as pv

hf_params = dict(
    Nx =  210,
    Ny = 330,
    tol = 1e-8,
    cores = None,
    inflationRate =  1.0015,
    metal = 'AISI406',
    itmaxSU2 = 1,
    GMSH_SOLVER = './gmsh',
    SU2_SOLVER = './SU2_CFD',
    baselineCP = './baselineCP.txt',
    T0in = 600.0,
    p0in = 8.0e5,
    itprintSU2 = 1,
    CP3_y =  -0.0024504689702769863,
    Thickness = 0.004209215007771409,
    rootfile = '../data/single_run/'
)


hf_fom = FOM()
hf_fom.set_run_model_function(hf_model)
hf_fom.set_params(hf_params)
hf_fom.set_variable_getter( get_su2_variable )
hf_fom.run()

In [ ]:
q1d_variables = [
    'Pressure',
    'Temperature',
    'Mach' 
]

design_variables = [
    'CP3_y',
    'Thickness' 
]

lf_snapshot = lf_fom.get_snapshot(variables=q1d_variables, design_variables=design_variables)

In [ ]:

def gen_snapshot(solution_path, variable_getter, variables, design_variables=None):
    data = {}
    for dvariable in design_variables:
        data[dvariable] = design_variables[dvariable]
    for variable_name in variables:
        variable_data = variable_getter(solution_path, variable_name)    
        data[variable_name] = variable_data

    return np.hstack([data[var] for var in data.keys()])
        
q1d_variables = [
    'Pressure',
    'Temperature',
    'Mach' 
]

design_variables = dict(
    CP3_y =  -0.0024504689702769863,
    Thickness = 0.004209215007771409
)

In [ ]:
snapshot = gen_snapshot(solution_path   ='../data/single_run/',
             variable_getter = get_q1d_variable, 
             variables       = q1d_variables,
             design_variables= design_variables)

In [ ]:
fr.reconstruct(lf_snapshot, 'Pressure').shape

In [ ]:
reconstructed = fr.reconstruct(lf_data_handler.test[:,0], 'Pressure')

In [ ]:
clear_mesh = clear_vtm(hf_fom.solution_file, hf_fom.su2infilepath + 'clean_mesh.vtm')

In [ ]:
mesh = pv.read(hf_fom.solution_file)

In [ ]:
clear_mesh

In [ ]:
def get_block_recursive(block, name_index):
    for n in name_index:
        block = block[n]
    return block

In [ ]:
def set_block_recursive(block, name_index, value):
    for n in name_index[:-1]:
        block = block[n]
    block[name_index[-1]] = value
    block.set_active_scalars(name_index[-1])
    return block

In [ ]:
block =  set_block_recursive(mesh, ['Zone 0 (Comp. Fluid)','Internal','Internal', 'Pressure'], reconstructed_pressure )

In [ ]:
block.plot(cpos='xy')

In [ ]:
mesh['Zone 0 (Comp. Fluid)']['Internal']['Internal'].set_active_scalars('Pressure')
mesh.plot(cpos='xy')

In [ ]:
original_mesh = pv.read('/home/ppiper/Dropbox/local/ihtc_repository/data/doe_30/28/SU2/outputs/cht_setupSU2.vtm')

In [ ]:
original_mesh['Zone 0 (Comp. Fluid)']['Internal']['Internal'].set_active_scalars('Pressure')
original_mesh['Zone 0 (Comp. Fluid)']['Internal']['Internal'].plot(cpos='xy')

In [ ]:
original_pressure = original_mesh['Zone 0 (Comp. Fluid)']['Internal']['Internal'][ 'Pressure']

In [ ]:
original_pressure.shape

In [ ]:
error = abs(original_pressure-reconstructed_pressure[:,0])/original_pressure

In [ ]:
error =  set_block_recursive(mesh, ['Zone 0 (Comp. Fluid)','Internal','Internal', 'Error'], error )

In [ ]:
error.plot(cpos='xy')